File is in early stages. Currently working on putting together the correct dataframe

In [1]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sqlite3
import seaborn as sns

In [2]:
sns.set(rc={'figure.figsize':(10,8)})

In [3]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

In [39]:
query = '''
 SELECT id, CAST(player_count AS INT64) player_count, player_recommendation rec, CAST(votes AS INT64) votes,
 SUM(CAST(votes AS INT64)) OVER (PARTITION BY id) game_votes,
 SUM(CAST(votes AS INT64)) OVER (PARTITION BY id, player_count) player_votes,
 
 -- column created for the sole purposes of ordering results
 CASE WHEN player_recommendation = 'Not Recommended' THEN 0
      WHEN player_recommendation = 'Recommended' THEN 1
      WHEN player_recommendation = 'Best' THEN 2
      ELSE NULL END order_col
 
 FROM BGG_PLAYER_RATINGS t1
 
 WHERE 1=1
     -- Remove the player counts that aren't possible, eg "5+"
     AND player_count NOT LIKE '%+%'
 
 ORDER BY id ASC, player_count ASC, order_col ASC
'''

results = cur.execute(query)
result_li = [elem for elem in results.fetchall()]
result_df = pd.DataFrame(result_li)
result_df.columns = ['id', 'players', 'rec', 'votes', 'game_votes', 'player_votes', 'temp']
result_df.drop('temp', axis = 1, inplace=True)

In [45]:
result_df['vote_percent'] = result_df['votes'] / result_df['player_votes']

In [46]:
# Checking query with Gloomhaven
result_df[result_df['id'] == '174430']

,id,players,rec,votes,game_votes,player_votes,temp,vote_percent
4404,174430,1,Not Recommended,270,3927,916,0,0.294760
4405,174430,1,Recommended,521,3927,916,1,0.568777
4406,174430,1,Best,125,3927,916,2,0.136463
4407,174430,2,Not Recommended,67,3927,1024,0,0.065430
4408,174430,2,Recommended,671,3927,1024,1,0.655273
4409,174430,2,Best,286,3927,1024,2,0.279297
4410,174430,3,Not Recommended,30,3927,1011,0,0.029674
4411,174430,3,Recommended,401,3927,1011,1,0.396637
4412,174430,3,Best,580,3927,1011,2,0.573689
4413,174430,4,Not Recommended,137,3927,976,0,0.140369


In [49]:
# Testing to see if the math works
assert 270 + 521 + 125 == 916

In [52]:
# Checking a game with no official single player
result_df[result_df['id'] == '246192']

,id,players,rec,votes,game_votes,player_votes,temp,vote_percent
8646,246192,1,Not Recommended,53,303,54,0,0.981481
8647,246192,1,Recommended,1,303,54,1,0.018519
8648,246192,1,Best,0,303,54,2,0.000000
8649,246192,2,Not Recommended,2,303,88,0,0.022727
8650,246192,2,Recommended,48,303,88,1,0.545455
8651,246192,2,Best,38,303,88,2,0.431818
8652,246192,3,Not Recommended,1,303,86,0,0.011628
8653,246192,3,Recommended,34,303,86,1,0.395349
8654,246192,3,Best,51,303,86,2,0.593023
8655,246192,4,Not Recommended,5,303,75,0,0.066667


In [54]:
# Checking how few votes a game might have
result_df[result_df['game_votes'] == result_df['game_votes'].min()]

,id,players,rec,votes,game_votes,player_votes,temp,vote_percent
3864,168917,1,Not Recommended,6,15,6,0,1.000000
3865,168917,1,Recommended,0,15,6,1,0.000000
3866,168917,1,Best,0,15,6,2,0.000000
3867,168917,2,Not Recommended,1,15,9,0,0.111111
3868,168917,2,Recommended,7,15,9,1,0.777778
3869,168917,2,Best,1,15,9,2,0.111111
